In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cmath    # Provides mathematical functions of a complex variable.

# Método de Laguerre

## $ \S 1 $ Introdução

Como antes, seja
$$
p\colon \mathbb C \to \mathbb C, \quad
p(z) = c_nz^n + c_{n - 1}z^{n - 1} + \cdots + c_1 z + c_0 \qquad
(c_k \in \mathbb C)
$$ 
um polinômio com coeficientes complexos. Do teorema fundamental da álgebra,
sabemos que $ p $ possui exatamente $ n $ raízes sobre $ \mathbb C $
(contadas com suas respectivas multiplicidades).  Neste caderno apresentaremos
um método iterativo específico para aproximação de zeros (inclusive complexos)
de polinômios deste tipo, conhecido como _método de Laguerre_.  Quando combinado
com a deflação polinomial estudada no caderno anterior, ele nos ajudará a
conseguir estimativas para _todos_ os zeros.

O método de Laguerre quase sempre converge a um zero, não importa qual seja a
estimativa inicial. Além do mais, sua convergência normalmente é ainda mais
rápida do que a do método de Newton. Mais precisamente, para estimativas
iniciais próximas o suficiente de um zero $ \alpha $ _simples_, a taxa de convergência
é cúbica, no sentido que o erro na estimativa atual é aproximadamente
proporcional ao cubo do erro envolvido na estimativa anterior. Se
$ \alpha $ é um zero múltiplo, a taxa de convergência é apenas linear.

O método de Laguerre é peculiar porque a teoria por trás dele não é muito
bem desenvolvida. Em particular, como já mencionado, ele não é infalível,
apesar de não serem conhecidas as condições precisas em que ele pode falhar.

## $ \S 2 $ Descrição do método de Laguerre

Seguindo [Kiusalaas], vamos derivar o método de Laguerre apenas no caso especial
em que o polinômio 
$$
p(z) = c_nz^n + c_{n - 1}z^{n - 1} + \cdots + c_1 z + c_0 \qquad
$$
possui um zero simples em $ \alpha $ e outro zero de multiplicidade $ n - 1 $ em
$ \beta $. Contudo, as fórmulas que obteremos funcionam bem mesmo no caso
geral. Sob estas hipóteses,
\begin{equation*}\label{E:zeta}
p(z) = (z - \alpha)(z - \beta)^{n - 1}\,. \tag{1}
\end{equation*}
Gostaríamos de aproximar $ \alpha $ a partir dos coeficientes $ c_k $.

Diferenciando \eqref{E:zeta}, deduzimos que
\begin{alignat*}{9}
p'(z) &= (z - \beta)^{n - 1} + (n - 1)(z - \alpha)(z - \beta)^{n - 2} \\
&= \bigg(\frac{1}{z - \alpha} + \frac{n - 1}{z - \beta} \bigg)\,.
\end{alignat*}
Portanto
$$
\frac{p'(z)}{p(z)} = \frac{1}{z - \alpha} + \frac{n - 1}{z - \beta}\,.
$$
Diferenciando esta última equação mais uma vez, 
$$
\frac{p''(z)}{p(z)} - \bigg(\frac{p'(z)}{p(z)}\bigg)^2 =
-\frac{1}{(z - \alpha)^2} - \frac{n - 1}{(z - \beta)^2}\,.
$$

Agora introduzimos os polinômios
\begin{equation*}\label{E:GH}
    \boxed{G(z) = \frac{p'(z)}{p(z)} \quad \text{e} \quad H(z) = G^2(z) - \frac{p''(z)}{p(z)}}
\tag{2}
\end{equation*}
Então as equações acima se tornam:
\begin{alignat*}{9}
    G(z) &= \frac{1}{n-1} \left(\frac{1}{z - \alpha} + \frac{1}{z - \beta}\right) \\
    H(z) &= \frac{1}{n-1} \left(\frac{1}{(z - \alpha)^2} + \frac{1}{(z - \beta)^2}\right)
\end{alignat*}
Resolvendo a primeira destas equações para $ z - \beta $ e substituindo a expressão
resultante na segunda, obtemos uma equação quadrática em $ z - \alpha $.
A solução dessa equação é a __fórmula de Laguerre__:
$$
\boxed{z - \alpha = \frac{n}{G(z) \pm \sqrt{(n - 1)\big[nH(z) - G^2(z)\big]}}}
$$
para $ G $ e $ H $ como em \eqref{E:GH}.

Agora voltando ao caso geral de um polinômio qualquer de grau $ n $,
o __método de Laguerre__ para se aproximar um zero de um polinômio usando a
fórmula de Laguerre é o seguinte:

In [ ]:
def horner_deriv(coefs: list[complex], z: complex
                 ) -> tuple[complex, complex, complex]:
    """
    Given a polynomial p represented by the list of its coefficients
    [c_0, c_1, ..., c_n] (where c_k is the coefficient of z^k) and a number z,
    evaluates the polynomial, its first derivative, and its second derivative
    at the point z and returns the corresponding values (y, dy, ddy).
    """
    n = len(coefs) - 1
    y = coefs.pop()     # Initial value for the polynomial.
    dy = 0.0 + 0.0j     # Initial value for the first derivative.
    ddy = 0.0 + 0.0j    # Initial value for the second derivative.

    # Iterate through the coefficients, updating the polynomial and its
    # derivatives at each step:
    for _ in range(n):
        ddy = ddy * z + 2.0 * dy
        dy = dy * z + y
        y = y * z + coefs.pop()

    return y, dy, ddy


def deflation(coefs: list[complex], r: complex) -> list[complex]:
    """
    Deflates a polynomial p represented by its list of coefficients `coefs`
    (from lowest to highest degree) by dividing it by a linear factor (z - r),
    where r is a complex root of the polynomial. Returns a list of complex
    numbers, representing the coefficients of the deflated polynomial.
    """
    n = len(coefs) - 1           # n = degree of p.
    new_coefs = [0] * n          # Initialize the list of new coefficients.
    new_coefs[n - 1] = coefs[n]  # Set the first new coefficient.
    
    # Perform the deflation algorithm:
    for k in range(n - 1, 0, -1):
        new_coefs[k - 1] = coefs[k] + r * new_coefs[k]
    return new_coefs

In [4]:
from random import random


def laguerre(coefs: list[complex], eps: float, max_iter: int = 30) -> complex:
    """
    Given a polynomial represented by the list of its coefficients, finds an
    approximate complex root of the polynomial using Laguerre's method.
    Parameters:
        * coefs: List of coefficients of the polynomial, in increasing degree.
        * eps: The tolerance for the error in the approximation of the roots.
        * max_iter: The maximum number of iterations performed.
    Returns:
        * A complex number that is an approximate root of the polynomial.
    Raises:
        * RuntimeError: If the tolerance is not achieved within
          max_iter iterations.
    """
    x = random()         # Starting value (random number).
    n = len(coefs) - 1   # Degree of the polynomial.

    for _ in range(max_iter):
        # Compute p and its first two derivatives at x:
        p, dp, ddp = horner_deriv(coefs, x)
        if abs(p) < eps:
            return x
        g = dp / p
        h = g * g - ddp / p
        # Take the (possibly complex) square root in Laguerre's formula:
        f = cmath.sqrt((n - 1) * (n * h - g * g))
        if abs(g + f) > abs(g - f):
            dx = n / (g + f)
        else:
            dx = n / (g - f)
        x = x - dx
        if abs(dx) < eps:
            return x

    raise RuntimeError("The maximum number of iterations was exceeded!")


In [3]:
def find_roots(coefs: list[complex], tol: float, max_iter: int = 30
               ) -> list[complex]:
    """
    Given a polynomial represented by the list of its coefficients, finds an
    approximations for all of its roots (including the complex ones)
    using a combination of Laguerre's method and deflation.
    Parameters:
        * coefs: List of coefficients of the polynomial, in increasing degree.
        * eps: The tolerance for the error in the approximation of the roots.
        * max_iter: The maximum number of iterations performed.
    Returns:
        * A list of complex numbers representing the approximate roots.
    """
    n = len(coefs) - 1    # n is the degree of p.
    roots = np.zeros((n), dtype=complex)    # Initialize the array of roots.
    
    # Iterate through the polynomial's degree, finding one root at a time:
    for k in range(n):
        z = laguerre(coefs, tol)
        # If the imaginary part of the root is small, consider it a real root:
        if abs(z.imag) < tol:
            z = z.real
        roots[k] = z
        # Deflate the polynomial using the root that was found:
        coefs = deflation(coefs, z)
    
    return roots

__Problema 1:__ Usando o método de Laguerre, encontre todas as raízes dos polinômios seguintes:

(a) $ p(z) = z^4 + 2.1z^3 - 2.52z^2 + 2.1z - 3.52. $

(b) $ p(z) = z^5 - 156z^4 - 5z^3 + 780z^2 + 4z - 624. $

(c) $ p(z) = z^6 + 4z^5 - 8z^4 - 34z^3 + 57z^2 + 130z - 150. $

(d) $ p(z) = 8z^7 + 28z^6 + 34z^5 - 13z^4 - 124z^3 + 19z^2 + 220z - 100. $

(e) $ p(z) = 2z^3 - 6(1 + i)z^2 + z - 6(1 - i). $

(f) $ p(z) = z^4 + (5 + i)z^3 - (8 - 5i)z^2 + (30 - 14i)z - 84. $
